In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
from datetime import datetime
from sklearn.linear_model import SGDClassifier
import sklearn.cross_validation
from sklearn.preprocessing import MinMaxScaler
import sklearn.metrics
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import xgboost
from vowpalwabbit.sklearn_vw import VWClassifier
import langdetect as ld
#from difflib import SequenceMatcher
from fuzzywuzzy import fuzz
import warnings 
warnings.filterwarnings("ignore")

/Library/Python/2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
SOURCE_TRAIN = "../data/train3_updated.csv"
df = pd.read_csv(SOURCE_TRAIN, sep=',')

SOURCE_VALID = "../data/valid3_updated.csv"
df_val = pd.read_csv(SOURCE_VALID, sep=',')

# Train + valid data in one
# SOURCE_TRAIN_ALL = "../data/train_final2.csv"
# df = pd.read_csv(SOURCE_TRAIN_ALL, sep=',')

# SOURCE_TEST = "../data/test3_updated.csv"
# df_test = pd.read_csv(SOURCE_TEST, sep=',')

In [34]:
df.head()

,Position,HistCTR,IsUserLoggedOn,Price,SearchCategoryLevel,reg_ads,highlighted_ads,context_ads,user_clicks_no,user_non_clicks_no,user_click_prob,day_of_week,hour,AdTitleSZ,SearchParamsSZ,SearchQuerySZ,weight,IsClick
0,7,0.005422,0,1000.0,3,3,0,2,5,15,0.250000,5,22,93,0,0,1,1
1,1,0.012844,0,77170.0,3,3,0,2,5,15,0.250000,5,22,66,0,0,1,0
2,1,0.002102,0,1790.0,3,2,0,2,5,21,0.192308,3,10,39,84,0,1,0
3,7,0.013889,0,3000.0,3,2,0,2,5,21,0.192308,3,10,48,84,0,1,1
4,1,0.033885,1,5470.0,3,4,0,1,1,7,0.125000,3,10,65,59,15,1,1


In [35]:
df.shape

(5766142, 18)

In [3]:
median_Price = df.Price.median()
df.Price.fillna(median_Price, inplace=True)

In [55]:
df.describe()

,Position,HistCTR,IsUserLoggedOn,Price,SearchCategoryLevel,reg_ads,highlighted_ads,context_ads,user_clicks_no,user_non_clicks_no,user_click_prob,day_of_week,hour,AdTitleSZ,SearchParamsSZ,SearchQuerySZ,weight,IsClick
count,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06
mean,3.594031e+00,1.125062e-02,3.144672e-01,1.842276e+04,2.693287e+00,1.702706e+00,3.985022e-01,1.863855e+00,2.537840e+00,1.437796e+01,1.966040e-01,2.867462e+00,1.459326e+01,5.101086e+01,2.292443e+01,2.613030e+00,3.321266e+01,1.966040e-01
std,2.972405e+00,1.593798e-02,4.643034e-01,9.228073e+04,5.958805e-01,1.303716e+00,5.079587e-01,3.429443e-01,3.858880e+00,2.413330e+01,1.980426e-01,1.977061e+00,5.775873e+00,1.994528e+01,2.966364e+01,7.221760e+00,2.325436e+01,3.974304e-01
min,1.000000e+00,1.000000e-05,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00
25%,1.000000e+00,2.715000e-03,0.000000e+00,1.300000e+03,3.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00,0.000000e+00,3.000000e+00,0.000000e+00,1.000000e+00,1.100000e+01,3.600000e+01,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00
50%,1.000000e+00,6.100000e-03,0.000000e+00,4.290000e+03,3.000000e+00,2.000000e+00,0.000000e+00,2.000000e+00,1.000000e+00,6.000000e+00,1.578947e-01,3.000000e+00,1.500000e+01,5.200000e+01,0.000000e+00,0.000000e+00,5.000000e+01,0.000000e+00
75%,7.000000e+00,1.299900e-02,1.000000e+00,1.330000e+04,3.000000e+00,3.000000e+00,1.000000e+00,2.000000e+00,3.000000e+00,1.600000e+01,3.333333e-01,5.000000e+00,1.900000e+01,6.600000e+01,4.200000e+01,0.000000e+00,5.000000e+01,0.000000e+00
max,7.000000e+00,1.000000e+00,1.000000e+00,1.040000e+07,3.000000e+00,4.000000e+00,4.000000e+00,2.000000e+00,1.070000e+02,5.960000e+02,1.000000e+00,6.000000e+00,2.300000e+01,9.700000e+01,1.980000e+02,1.620000e+02,5.000000e+01,1.000000e+00


In [56]:
df.dtypes

Position                 int64
HistCTR                float64
IsUserLoggedOn           int64
Price                  float64
SearchCategoryLevel      int64
reg_ads                  int64
highlighted_ads          int64
context_ads              int64
user_clicks_no           int64
user_non_clicks_no       int64
user_click_prob        float64
day_of_week              int64
hour                     int64
AdTitleSZ                int64
SearchParamsSZ           int64
SearchQuerySZ            int64
weight                   int64
IsClick                  int64
dtype: object

In [4]:
labels = df["IsClick"]
labels.head()

0    1
1    0
2    0
3    1
4    1
Name: IsClick, dtype: int64

In [5]:
weights = df["weight"]
weights.head()
weights.value_counts()

50    3386272
1     1764726
Name: weight, dtype: int64

In [6]:
df = df.drop(["IsClick"], axis=1)
df = df.drop(["weight"], axis=1)
df.head()

,Position,HistCTR,IsUserLoggedOn,Price,SearchCategoryLevel,reg_ads,highlighted_ads,context_ads,user_clicks_no,user_non_clicks_no,user_click_prob,day_of_week,hour,AdTitleSZ,SearchParamsSZ,SearchQuerySZ
0,7,0.005422,0,1000.0,3,3,0,2,5,15,0.250000,5,22,93,0,0
1,1,0.012844,0,77170.0,3,3,0,2,5,15,0.250000,5,22,66,0,0
2,1,0.002102,0,1790.0,3,2,0,2,5,21,0.192308,3,10,39,84,0
3,7,0.013889,0,3000.0,3,2,0,2,5,21,0.192308,3,10,48,84,0
4,1,0.033885,1,5470.0,3,4,0,1,1,7,0.125000,3,10,65,59,15


In [16]:
scaler = MinMaxScaler(feature_range=(0, 1), copy=True)

#Selected initially
df.Price = scaler.fit_transform(df.Price.reshape(-1, 1))
df.AdTitleSZ = scaler.fit_transform(df.AdTitleSZ.reshape(-1, 1))
df.SearchParamsSZ = scaler.fit_transform(df.SearchParamsSZ.reshape(-1, 1))
df.SearchQuerySZ = scaler.fit_transform(df.SearchQuerySZ.reshape(-1, 1))

#Rest
# df.Position = scaler.fit_transform(df.Position.reshape(-1, 1))
# df.HistCTR = scaler.fit_transform(df.HistCTR.reshape(-1, 1))
# df.SearchCategoryLevel = scaler.fit_transform(df.SearchCategoryLevel.reshape(-1, 1))
# df.reg_ads = scaler.fit_transform(df.reg_ads.reshape(-1, 1))
# df.highlighted_ads = scaler.fit_transform(df.highlighted_ads.reshape(-1, 1))
# df.context_ads = scaler.fit_transform(df.context_ads.reshape(-1, 1))
# df.user_clicks_no = scaler.fit_transform(df.user_clicks_no.reshape(-1, 1))
# df.user_non_clicks_no = scaler.fit_transform(df.user_non_clicks_no.reshape(-1, 1))
# df.day_of_week = scaler.fit_transform(df.day_of_week.reshape(-1, 1))
# df.hour = scaler.fit_transform(df.hour.reshape(-1, 1))

In [74]:
weights.head()

0    1.0
1    1.0
2    1.0
3    1.0
4    1.0
Name: weight, dtype: float64

In [76]:
df.describe()

,Position,HistCTR,IsUserLoggedOn,Price,SearchCategoryLevel,reg_ads,highlighted_ads,context_ads,user_clicks_no,user_non_clicks_no,user_click_prob,day_of_week,hour,AdTitleSZ,SearchParamsSZ,SearchQuerySZ
count,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06
mean,3.594031e+00,1.125062e-02,3.144672e-01,1.771419e-03,2.693287e+00,1.702706e+00,3.985022e-01,1.863855e+00,2.537840e+00,1.437796e+01,1.966040e-01,2.867462e+00,1.459326e+01,5.258851e-01,1.157799e-01,1.612981e-02
std,2.972405e+00,1.593798e-02,4.643034e-01,8.873147e-03,5.958805e-01,1.303716e+00,5.079587e-01,3.429443e-01,3.858880e+00,2.413330e+01,1.980426e-01,1.977061e+00,5.775873e+00,2.056215e-01,1.498164e-01,4.457877e-02
min,1.000000e+00,1.000000e-05,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.000000e+00,2.715000e-03,0.000000e+00,1.250000e-04,3.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00,0.000000e+00,3.000000e+00,0.000000e+00,1.000000e+00,1.100000e+01,3.711340e-01,0.000000e+00,0.000000e+00
50%,1.000000e+00,6.100000e-03,0.000000e+00,4.125000e-04,3.000000e+00,2.000000e+00,0.000000e+00,2.000000e+00,1.000000e+00,6.000000e+00,1.578947e-01,3.000000e+00,1.500000e+01,5.360825e-01,0.000000e+00,0.000000e+00
75%,7.000000e+00,1.299900e-02,1.000000e+00,1.278846e-03,3.000000e+00,3.000000e+00,1.000000e+00,2.000000e+00,3.000000e+00,1.600000e+01,3.333333e-01,5.000000e+00,1.900000e+01,6.804124e-01,2.121212e-01,0.000000e+00
max,7.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,3.000000e+00,4.000000e+00,4.000000e+00,2.000000e+00,1.070000e+02,5.960000e+02,1.000000e+00,6.000000e+00,2.300000e+01,1.000000e+00,1.000000e+00,1.000000e+00


## Validation set

In [8]:
df_val.describe()

,Position,HistCTR,IsUserLoggedOn,Price,SearchCategoryLevel,reg_ads,highlighted_ads,context_ads,user_clicks_no,user_non_clicks_no,user_click_prob,day_of_week,hour,AdTitleSZ,SearchParamsSZ,SearchQuerySZ,weight,IsClick
count,615144.000000,615144.000000,615144.000000,6.147900e+05,615144.000000,615144.000000,615144.000000,615144.000000,615144.000000,615144.000000,615144.000000,615144.000000,615144.000000,615144.000000,615144.000000,615144.000000,615144.000000,615144.000000
mean,3.697476,0.013614,0.233051,2.144333e+04,2.654273,1.706074,0.429152,1.899038,0.816510,4.959044,0.080637,2.431881,14.076862,51.561140,21.281133,3.320075,31.014914,0.217156
std,2.984710,0.017969,0.422775,4.776319e+05,0.644539,1.256227,0.515412,0.301278,2.354575,16.148479,0.171698,2.537848,5.534603,19.596491,28.338633,8.211384,23.871252,0.412310
min,1.000000,0.000010,0.000000,0.000000e+00,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,1.000000,0.003214,0.000000,1.490000e+03,2.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,10.000000,38.000000,0.000000,0.000000,1.000000,0.000000
50%,1.000000,0.007245,0.000000,4.790000e+03,3.000000,2.000000,0.000000,2.000000,0.000000,0.000000,0.000000,1.000000,14.000000,52.000000,0.000000,0.000000,50.000000,0.000000
75%,7.000000,0.016858,0.000000,1.299000e+04,3.000000,3.000000,1.000000,2.000000,1.000000,3.000000,0.058824,6.000000,18.000000,66.000000,41.000000,0.000000,50.000000,0.000000
max,7.000000,0.891552,1.000000,1.300000e+08,3.000000,4.000000,4.000000,2.000000,107.000000,596.000000,1.000000,6.000000,23.000000,97.000000,181.000000,85.000000,50.000000,1.000000


In [7]:
median_Price = df_val.Price.median()
df_val.Price.fillna(median_Price, inplace=True)

In [8]:
labels_val = df_val["IsClick"]
labels_val.head()

15    0
16    1
28    0
29    1
36    1
Name: IsClick, dtype: int64

In [9]:
weights_val = df_val["weight"]
weights_val.head()
weights_val.value_counts()

50    376806
1     238338
Name: weight, dtype: int64

In [10]:
df_val = df_val.drop(["IsClick"], axis=1)
df_val = df_val.drop(["weight"], axis=1)
df_val.head()

,Position,HistCTR,IsUserLoggedOn,Price,SearchCategoryLevel,reg_ads,highlighted_ads,context_ads,user_clicks_no,user_non_clicks_no,user_click_prob,day_of_week,hour,AdTitleSZ,SearchParamsSZ,SearchQuerySZ
15,7,0.009569,1,1342.0,3,3,0,2,1,1,0.5,6,21,48,0,0
16,1,0.011035,1,2500.0,3,3,0,2,1,1,0.5,6,21,41,0,0
28,7,0.005291,0,1000.0,3,3,0,2,0,0,0.0,6,18,47,24,0
29,1,0.031800,0,1409.0,3,3,0,2,0,0,0.0,6,18,50,24,0
36,7,0.008260,0,15990.0,3,2,1,2,0,2,0.0,6,15,27,0,0


In [17]:
#Selected initially
df_val.Price = scaler.fit_transform(df_val.Price.reshape(-1, 1))
df_val.AdTitleSZ = scaler.fit_transform(df_val.AdTitleSZ.reshape(-1, 1))
df_val.SearchParamsSZ = scaler.fit_transform(df_val.SearchParamsSZ.reshape(-1, 1))
df_val.SearchQuerySZ = scaler.fit_transform(df_val.SearchQuerySZ.reshape(-1, 1))

#Rest
# df_val.Position = scaler.fit_transform(df_val.Position.reshape(-1, 1))
# df_val.HistCTR = scaler.fit_transform(df_val.HistCTR.reshape(-1, 1))
# df_val.SearchCategoryLevel = scaler.fit_transform(df_val.SearchCategoryLevel.reshape(-1, 1))
# df_val.reg_ads = scaler.fit_transform(df_val.reg_ads.reshape(-1, 1))
# df_val.highlighted_ads = scaler.fit_transform(df_val.highlighted_ads.reshape(-1, 1))
# df_val.context_ads = scaler.fit_transform(df_val.context_ads.reshape(-1, 1))
# df_val.user_clicks_no = scaler.fit_transform(df_val.user_clicks_no.reshape(-1, 1))
# df_val.user_non_clicks_no = scaler.fit_transform(df_val.user_non_clicks_no.reshape(-1, 1))
# df_val.day_of_week = scaler.fit_transform(df_val.day_of_week.reshape(-1, 1))
# df_val.hour = scaler.fit_transform(df_val.hour.reshape(-1, 1))

## Test data

In [66]:
df_test.describe()

,ID,Position,HistCTR,IsUserLoggedOn,Price,SearchCategoryLevel,reg_ads,highlighted_ads,context_ads,user_clicks_no,user_non_clicks_no,user_click_prob,day_of_week,hour,AdTitleSZ,SearchParamsSZ,SearchQuerySZ
count,7.816361e+06,7.816361e+06,7.816361e+06,7.816361e+06,7.814321e+06,7.816361e+06,7.816361e+06,7.816361e+06,7.816361e+06,7.816361e+06,7.816361e+06,7.816361e+06,7.816361e+06,7.816361e+06,7.816361e+06,7.816361e+06,7.816361e+06
mean,3.908180e+06,3.702706e+00,1.631742e-02,1.793923e-01,2.184264e+04,2.452776e+00,1.256610e+00,5.708090e-01,1.900901e+00,2.630371e-01,1.079616e+00,5.656097e-02,2.558602e+00,1.433917e+01,5.388597e+01,1.561031e+01,2.814991e+00
std,2.256389e+06,2.985233e+00,2.009643e-02,3.836805e-01,1.846800e+05,6.691422e-01,1.294963e+00,5.103030e-01,2.987948e-01,9.433144e-01,3.975266e+00,1.639801e-01,1.904586e+00,5.518377e+00,1.903151e+01,2.633706e+01,7.477953e+00
min,0.000000e+00,1.000000e+00,1.000000e-05,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.954090e+06,1.000000e+00,3.696000e-03,0.000000e+00,1.300000e+03,2.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.100000e+01,4.100000e+01,0.000000e+00,0.000000e+00
50%,3.908180e+06,1.000000e+00,8.260000e-03,0.000000e+00,4.820000e+03,3.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00,1.500000e+01,5.600000e+01,0.000000e+00,0.000000e+00
75%,5.862270e+06,7.000000e+00,2.424900e-02,0.000000e+00,1.300000e+04,3.000000e+00,2.000000e+00,1.000000e+00,2.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,4.000000e+00,1.900000e+01,6.800000e+01,2.900000e+01,0.000000e+00
max,7.816360e+06,7.000000e+00,1.000000e+00,1.000000e+00,1.300000e+08,3.000000e+00,4.000000e+00,4.000000e+00,2.000000e+00,1.100000e+02,6.860000e+02,1.000000e+00,6.000000e+00,2.300000e+01,9.700000e+01,2.250000e+02,2.180000e+02


In [67]:
median_Price = df_test.Price.median()
df_test.Price.fillna(median_Price, inplace=True)

In [68]:
df_test.SearchCategoryLevel.fillna(0, inplace=True)

In [71]:
df_test = df_test.drop(["ID"], axis=1)

## Logistic Regression

In [18]:
from sklearn.linear_model import LogisticRegression

In [19]:
logreg = LogisticRegression()

In [20]:
logreg.fit(df, labels, sample_weight=weights)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [21]:
pred_probs = logreg.predict_proba(df_val)

# pred_probs = logreg.predict_proba(df_test)
# now jump to kaggle submission

In [22]:
# sklearn.metrics.log_loss(labels_val, pred_probs)

sklearn.metrics.log_loss(labels_val, pred_probs, sample_weight=weights_val)

0.060160204028136342

## Logistic Regression with Stochastic Gradient Descent

In [138]:
CLF_LogReg_SGF = SGDClassifier(loss="log")
print(CLF_LogReg_SGF)

TypeError: __init__() got an unexpected keyword argument 'intercept_scaling'

In [104]:
# CLF_LogReg_SGF.fit(df, labels, sample_weight=weights)
CLF_LogReg_SGF.fit(df, labels)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

In [106]:
sklearn.metrics.log_loss(labels_val, CLF_LogReg_SGF.predict_proba(df_val))

0.60123697546312083

## XGboost

In [ ]:
CLF_Xgboost = xgboost.XGBClassifier()
print(CLF_Xgboost)

In [ ]:
CLF_Xgboost.fit(inputs, labels)

In [ ]:
CLF_Xgboost.predict(inputs)
CLF_Xgboost.score(df_val, labels_val)

## Vowpal Wabbit

In [ ]:
# df["weight"] = df["weight"].replace(to_replace=50, value=1.5)

In [111]:
CLF_wabbit = VWClassifier(link="logistic", loss_function='logistic', quiet=False)
print(CLF_wabbit)

{'link': 'logistic', 'quiet': False, 'loss_function': 'logistic'}


In [112]:
CLF_wabbit.fit(df, labels)

{'link': 'logistic', 'quiet': False, 'loss_function': 'logistic'}

In [ ]:
# CLF_wabbit.fit(df, labels, sample_weight=weights)

In [114]:
# CLF_wabbit.predict(inputs)
# CLF_wabbit.score(df_val, labels_val)

predictions = CLF_wabbit.predict(df_test)

In [115]:
predictions

array([-1.,  1.,  1., ..., -1., -1.,  1.])

## Kaggle submission

In [108]:
pred_probs = np.delete(pred_probs, 0, 1) # Delete first column representing probabilities of ads not being clicked

In [109]:
pred_probs

array([[ 0.0039206 ],
       [ 0.02955382],
       [ 0.00109828],
       ..., 
       [ 0.01229868],
       [ 0.00575759],
       [ 0.00192226]])

In [110]:
sample_sub = pd.read_csv('../data/sampleSubmission.csv')
sample_sub = sample_sub.drop(["IsClick"], axis=1)
sample_sub.insert(1, 'IsClick', pred_probs)
sample_sub.to_csv('sub.csv' , index=False )

In [105]:
sample_sub.head()

,ID,IsClick
0,1,0.217511
1,5,0.217852
2,6,0.146813
3,7,0.125223
4,8,0.177037


In [91]:
sample_sub['ID'].shape 

(7816361,)

In [90]:
pred_probs.shape

(7816361, 1)

In [236]:
sample_sub.head()

,ID,IsClick
0,1,0
1,5,0
2,6,0
3,7,0
4,8,0
